In [1]:
import csv
import random

# Generate random 100-bit sequence
bit_sequence = [random.randint(0, 1) for _ in range(100)]

samples_per_bit = 10

# Expand bits into repeated samples
waveform = []
for bit in bit_sequence:
    waveform.extend([bit] * samples_per_bit)

# Save to CSV
with open("waveform.csv", "w", newline="") as file:
    writer = csv.writer(file)
    for sample in waveform:
        writer.writerow([sample])

print("Random 100-bit waveform saved as waveform.csv")


Random 100-bit waveform saved as waveform.csv


In [3]:
import csv
import random
import numpy as np

# 1. Generate random 100-bit sequence
bit_sequence = [random.randint(0,1) for _ in range(100)]
samples_per_bit = 10

# 2. Expand each bit into samples, convert to float
waveform = np.array([float(bit) for bit in bit_sequence for _ in range(samples_per_bit)])

# 3. Compute RMS voltage of the waveform
v_rms_signal = np.sqrt(np.mean(waveform**2))
print(f"Signal RMS voltage = {v_rms_signal:.5f}")

# Function to add AWGN at given SNR_dB
def add_awgn_and_rescale(signal, snr_db):
    # compute noise RMS
    v_rms_noise = v_rms_signal / (10**(snr_db / 20.0))
    print(f"For SNR={snr_db} dB → Noise RMS = {v_rms_noise:.5f}")
    # generate noise (Gaussian with zero mean)
    noise = np.random.randn(len(signal)) * v_rms_noise
    # add noise to signal
    noisy = signal + noise
    # rescale so max(|value|) = 1
    max_abs = np.max(np.abs(noisy))
    if max_abs == 0:
        rescaled = noisy
    else:
        rescaled = noisy / max_abs
    return rescaled

# 4. Define SNR values
snr_values = [24, 18, 12, 8, 6]

# 5. Generate noisy waveforms & save CSVs
for snr in snr_values:
    noisy_wave = add_awgn_and_rescale(waveform, snr)
    fname = f"waveform_SNR_{snr}dB.csv"
    with open(fname, "w", newline="") as f:
        writer = csv.writer(f)
        for v in noisy_wave:
            writer.writerow([v])
    print(f"Saved {fname}")


Signal RMS voltage = 0.67823
For SNR=24 dB → Noise RMS = 0.04279
Saved waveform_SNR_24dB.csv
For SNR=18 dB → Noise RMS = 0.08538
Saved waveform_SNR_18dB.csv
For SNR=12 dB → Noise RMS = 0.17036
Saved waveform_SNR_12dB.csv
For SNR=8 dB → Noise RMS = 0.27001
Saved waveform_SNR_8dB.csv
For SNR=6 dB → Noise RMS = 0.33992
Saved waveform_SNR_6dB.csv


In [4]:
import csv
import random
import numpy as np

# === 1. Generate random bitstream (200 bits) ===
bitstream = [random.randint(0, 1) for _ in range(200)]

# === 2. Group bits into pairs (PAM-4 symbols) ===
symbols = []
for i in range(0, len(bitstream), 2):
    b1, b2 = bitstream[i], bitstream[i+1]
    # Gray-coded PAM-4 mapping:
    if (b1, b2) == (0, 0): symbols.append(-3)
    if (b1, b2) == (0, 1): symbols.append(-1)
    if (b1, b2) == (1, 1): symbols.append(+1)
    if (b1, b2) == (1, 0): symbols.append(+3)

# === 3. Upsample: 10 samples per symbol ===
samples_per_symbol = 10
waveform = np.array([level for s in symbols for level in [s]*samples_per_symbol], dtype=float)

# === 4. Compute RMS of the clean PAM-4 signal ===
v_rms_signal = np.sqrt(np.mean(waveform**2))
print(f"Signal RMS: {v_rms_signal:.5f}")

# === 5. AWGN generation function ===
def add_awgn_and_rescale(signal, snr_db):
    # Noise RMS formula (voltage-based SNR):
    # SNR_dB = 20 * log10(Vrms_signal / Vrms_noise)
    v_rms_noise = v_rms_signal / (10**(snr_db / 20))

    # Create noise (Gaussian, zero-mean)
    noise = np.random.randn(len(signal)) * v_rms_noise

    noisy = signal + noise

    # Rescale so max(abs()) == 1
    max_abs = np.max(np.abs(noisy))
    if max_abs != 0:
        noisy = noisy / max_abs

    return noisy

# === 6. SNR values to test ===
snr_values = [24, 18, 12, 8, 6]

# === 7. Generate & save noisy CSVs ===
for snr in snr_values:
    noisy_data = add_awgn_and_rescale(waveform, snr)
    filename = f"pam4_SNR_{snr}dB.csv"
    with open(filename, "w", newline="") as f:
        writer = csv.writer(f)
        for v in noisy_data:
            writer.writerow([v])
    print(f"Saved {filename}")


Signal RMS: 2.30651
Saved pam4_SNR_24dB.csv
Saved pam4_SNR_18dB.csv
Saved pam4_SNR_12dB.csv
Saved pam4_SNR_8dB.csv
Saved pam4_SNR_6dB.csv
